In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

Using TensorFlow backend.


In [2]:
# Turn inFile into a one hot label
def createOneHot(inFile,numberOfClasses):
    outFile = np.zeros((inFile.shape[0],inFile.shape[1], numberOfClasses));
    for i in range(0,inFile.shape[0]):
        for j in range(0,inFile.shape[1]):
            outFile[i,j,inFile[i,j]] = 1;
    return outFile;

In [3]:
# Turn 41 orthomosaics into one array. 
def loadOneOrthomosaic(imagePath1,imagePath2):
    temp= imageio.imread(imagePath1 + '1.png');
    inputData = np.zeros( (temp.shape[0],temp.shape[1],inputChannels) )
    for i in range(1,25):
        temp= imageio.imread(imagePath1 + str(i) + '.png');
        inputData[:,:,i] = temp[:,:,0];
    for i in range(1,16):
        temp= imageio.imread(imagePath2 + str(i) + '.png');
        inputData[:,:,i+25] = temp[:,:,0];
    return np.expand_dims(inputData,axis=0)

In [9]:
def loadOneOrthomosaic(imagePath):
    temp= imageio.imread(imagePath1 + '1.png');
    inputData = np.zeros(( temp.shape[0], temp.shape[1], inputChannels))
    for i in range(1,41):
        temp= imageio.imread(imagePath1 + str(i) + '.png');
        inputData[:,:,i] = temp[:,:];
    return np.expand_dims( inputData, axis=0)

In [5]:
def loadOneLabel(imagePath):
    temp = sio.loadmat(imagePath);
    trainOneHotLabels =  createOneHot( temp['labeledPicture'], 9);
    return np.expand_dims( trainOneHotLabels, axis=0)

In [6]:
def readSet(paths):
    numberOfImages = len(paths);
    data = np.zeros( (numberOfImages,imageSize[1],imageSize[0],inputChannels) );
    labels = np.zeros((numberOfImages,imageSize[1],imageSize[0],amountOfClasses))
    for i in range(0,numberOfImages):
        data[i,:,:,:] = loadOneOrthomosaic(paths[i][0]);
        labels[i,:,:,:] = loadOneLabel(paths[i][1]);
    return data,labels

def makeDimensionsEven(inArray):
    if inArray.shape[0] % 2 != 0:
        middleArray = np.append(inArray, np.zeros((1,inArray.shape[1])), axis=0)
    else:
        middleArray = inArray
    if middleArray.shape[1] % 2 != 0:
        outArray = np.append(middleArray, np.zeros((middleArray.shape[0],1)), axis=1)
    else:
        outArray = middleArray;
    return outArray;

def loadData(pathArray):
    # each row of pathArray belongs to one dataset and has pathNIR, pathVIS, pathLabel
    #trainDataIn = np.load('trainingData.npy');
    outData = [];
    temp = returnOrthomosaicArray(pathArray[0,0],pathArray[0,1]);
    outData.append(temp) 

model = Sequential([
    Conv2D(64, 5, input_shape=inputDataShape, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 5, activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(128, 6, strides=2, padding='valid', activation='relu'),
    Conv2DTranspose(64, 6, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])

In [7]:
basePath = '/Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/';

imagePath1 = basePath + 'Data/FIP/20170622/Band';
labelpath1 = basePath + 'Labels/FIP20170622.mat';

imagePath2 = basePath + 'Data/Ximea_Tamron/20170510/Band';
labelpath2 = basePath + 'Labels/XimeaT20170510.mat';

imagePath3 = basePath + 'Data/Ximea_Tamron/20170622/Band';
labelpath3 = basePath + 'Labels/XimeaT20170622.mat';

trainPaths = [[imagePath1,labelpath1],[imagePath2,labelpath2],[imagePath3,labelpath3]];

amountOfClasses = 9;
inputChannels = 41;
imageSize = [1600,1600]; # [X,Y]

model = Sequential([
    Conv2D(20, 5, input_shape=(None, None, inputChannels), activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(20, 6, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])

model.compile(optimizer='adagrad',
              loss='categorical_crossentropy',
              metrics=['accuracy']);

In [10]:
# Load the training data and labels

trainData,trainLabels = readSet(trainPaths)


In [11]:
dir()

['Conv2D',
 'Conv2DTranspose',
 'Image',
 'In',
 'MaxPooling2D',
 'Out',
 'Sequential',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'amountOfClasses',
 'basePath',
 'createOneHot',
 'exit',
 'get_ipython',
 'imagePath1',
 'imagePath2',
 'imagePath3',
 'imageSize',
 'imageio',
 'inputChannels',
 'keras',
 'labelpath1',
 'labelpath2',
 'labelpath3',
 'loadOneLabel',
 'loadOneOrthomosaic',
 'model',
 'np',
 'plt',
 'quit',
 'readSet',
 'sio',
 'trainData',
 'trainLabels',
 'trainPaths']

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 20)    20520     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 20)    0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, None, None, 20)    14420     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 9)     189       
Total params: 35,129
Trainable params: 35,129
Non-trainable params: 0
_________________________________________________________________


In [14]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='acc', patience=2);

model.fit(trainData, trainLabels, epochs=1,callbacks=[early_stopping]);


Epoch 1/1
3/3 [==============================] - 225s 75s/step - loss: 3.3614 - acc: 0.7761


In [15]:
model.save('networkModels/FCN-None-None-conv5.h5')